In [8]:
pip install python3-discogs-client

You should consider upgrading via the 'c:\users\river\documents\transparence\discogs_project\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import discogs_client

In [2]:
d = discogs_client.Client('river_agent/1.1', user_token='qtIjahdmzFABMsZpAlcCOiwenwKlCJHzVJeAJoXt')

In [7]:
class MyTrack:
    """
    One Instance represents one row in the final excel table.
    """
    def __init__(self, title, album, posi):
        self.title = title
        self.track_artist = set()
        self.album = album
        self.labels = set()
        self.release_year = set()
        self.instruments = set()
        self.genre = set()
        self.format = set()
        self.countries_of_release = set()
        self.catalog_numbers = set()
        self.requires_manual_check = False
        self.id_str = title + " in " + album
        self.url = None
        self.position = posi
        self.track_specific_info = set()

In [4]:
class MyAlbum:
    def __init__(self, title):
        self.album_title = title
        self.track_specific_info = set()
        self.tracklist = []
        self.version_ids = set()

In [5]:
# 108475 is the artist id for Greg Phillinganes.
greg = d.artist(108475)
# All the releases/masters related to Greg Phillinganes.
greg_rel = greg.releases
# The release list can be paginated
greg_rel_page1 = greg_rel.page(0)
greg_rel_page5 = greg_rel.page(5)


In [5]:
# Final results will be stored in this dictionary
# Key: Each track's title
# Value: a MyTrack obj
track_dic = {}
# Name to be searched
artist_name = greg.name
# The artist instance
artist_obj = greg
# List of albums
album_list = []


def create_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name, cur_album):
    """
    Create a new MyTrack instance based on the track_entry in the cur_release.
    :param cur_album:
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: title of the new track
    """
    if main_release_name == "":
        main_release_name = cur_release.title.strip()
    new_key = track_entry.title.strip() + " in " + main_release_name
    track_tmp = MyTrack(track_entry.title, main_release_name, track_entry.position)
    track_tmp.url = cur_release.url
    for artist_entry in cur_release.artists:
        track_tmp.track_artist.add(artist_entry.name)
    for label_entry in cur_release.labels:
        track_tmp.labels.add(label_entry.name)
        track_tmp.catalog_numbers.add(label_entry.data['catno'])
    track_tmp.release_year.add(cur_release.year)
    if is_release_main_artist:
        track_tmp.instruments.add("Main")
    if is_in_credits:
        for extra_artist_entry in cur_release.data['extraartists']:
            if extra_artist_entry['name'] == artist_name:
                track_tmp.instruments.add(extra_artist_entry['role'])
                if extra_artist_entry['tracks'] != "":
                    track_tmp.requires_manual_check = True
                    track_tmp.track_specific_info.add(extra_artist_entry['tracks'])
                    cur_album.track_specific_info.add(extra_artist_entry['tracks'])
                break
    track_tmp.genre.update(cur_release.genres)
    for format_entry in cur_release.formats:
        track_tmp.format.add(format_entry['name'])
    track_tmp.countries_of_release.add(cur_release.country)
    track_tmp.duration = track_entry.duration
    if artist_obj in track_entry.credits:
        new_role = ""
        for ex_artist_entry in track_entry.data['extraartists']:
            if ex_artist_entry['name'] == artist_name:
                new_role = ex_artist_entry['role']
        track_tmp.instruments.add(new_role)
    track_dic[new_key] = track_tmp
    cur_album.tracklist.append(track_tmp)
    return track_entry.title


def update_a_mytrack(cur_release, track_entry, main_release_name):
    """
    Update an existing MyTrack instance in the track_dic, based on the track_entry in the cur_release.
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param main_release_name: the title of the main_release of the master
    :return: the title of the updated MyTrack instance
    """
    if main_release_name != "":
        new_key = track_entry.title.strip() + " in " + main_release_name
    else:
        new_key = track_entry.title.strip() + " in " + cur_release.title.strip()
    track_cur = track_dic.get(new_key)

    for label_entry in cur_release.labels:
        track_cur.labels.add(label_entry.name)
        track_cur.catalog_numbers.add(label_entry.data['catno'])
    track_cur.countries_of_release.add(cur_release.country)
    track_cur.release_year.add(cur_release.year)
    for format_entry in cur_release.formats:
        track_cur.format.add(format_entry['name'])
    return track_entry.title


def check_single_track_credit(cur_release, track_entry, artist_t, is_release_main_artist, is_in_credits, main_release_name):
    """
    If the artist_obj is not the main artist of the release, then check artist_obj's contribution in a single track.
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param artist_t: artist instance to be found
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: title of the current track, whether updated
    """
    new_key = track_entry.title.strip() + " in " + main_release_name
    if artist_t in track_entry.credits:
        if new_key in track_dic.keys():
            cur_track_title = update_a_mytrack(cur_release, track_entry, main_release_name)
        else:
            cur_track_title = create_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name)
        new_role = ""
        for ex_artist_entry in track_entry.data['extraartists']:
            if ex_artist_entry['name'] == artist_name:
                new_role = ex_artist_entry['role']
        track_dic[new_key].instruments.add(new_role)
        return cur_track_title, True
    return "", False


def get_info_by_single_track(cur_release, is_release_main_artist, is_in_credits, track_title_list, main_release_name):
    """
    If the artist_obj is not the main artist of the release, then check artist_obj's contribution track by track.
    :param track_title_list: list of created MyTrack instances' titles
    :param cur_release: current version of the master
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: null
    """
    for track_e in cur_release.tracklist:
        cur_track_title, is_updated = check_single_track_credit(cur_release, track_e, artist_obj, is_release_main_artist, is_in_credits, main_release_name)
        if is_updated:
            track_title_list.append(cur_track_title)


def get_info_from_release(cur_release, main_release_name):
    """
    Complete data retrieving in one version/release.
    :param cur_release: current version of the master
    :param main_release_name: the title of the main_release of the master
    :return: list of created MyTrack instances' titles, whether current artist is the main artist || current artist appear in "Credits"
    """
    is_release_main_artist = False
    is_in_credits = False
    track_title_list = []
    cur_album = MyAlbum(cur_release.title.strip())
    cur_album.version_ids.add(cur_release.id)
    if greg in cur_release.artists:
        is_release_main_artist = True
    if greg in cur_release.credits:
        is_in_credits = True
    if is_release_main_artist or is_in_credits:
        for track_entry in cur_release.tracklist:
            new_key = track_entry.title.strip() + " in " + cur_release.title.strip()
            if new_key in track_dic.keys():
                update_a_mytrack(main_release_tmp, track_entry, main_release_name)
            else:
                track_title_list.append(create_new_mytrack(main_release_tmp, track_entry, is_release_main_artist, is_in_credits, main_release_name))

        return track_title_list, True
    else:
        get_info_by_single_track(cur_release, is_release_main_artist, is_in_credits, track_title_list, main_release_name)
        return track_title_list, False

def update_from_other_release_version(cur_release, track_title_list, update_all, main_release_name, cur_album):
    """
    Update MyTrack in versions which are not main_release/version.
    :param cur_album:
    :param cur_release: current version of the master
    :param track_title_list: list of created MyTrack instances' titles
    :param update_all: whether current artist is the main artist || current artist appear in "Credits"
    :param main_release_name: the title of the main_release of the master
    :return: null
    """
    if cur_release.id in cur_album.version_ids:
        return
    else:
        cur_album.version_ids.add(cur_release.id)
    is_release_main_artist = False
    is_in_credits = False
    if greg in cur_release.artists:
        is_release_main_artist = True
    if greg in cur_release.credits:
        is_in_credits = True
    for track_entry in cur_release.tracklist:
        if track_entry.title in track_title_list:
            # print(track_title_list)
            update_a_mytrack(cur_release, track_entry, main_release_name)
        elif update_all:
            track_title_list.append(create_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name))
        check_single_track_credit(cur_release, track_entry, artist_obj, is_release_main_artist, is_in_credits, main_release_name)

# Main program starts here
for cur_rel in greg_rel_page5:
    # Each release's type is other master or release
    if cur_rel.data["type"] == "master":
        main_release_tmp = cur_rel.main_release
        # We first check the main_release of the master
        track_names, should_update_all = get_info_from_release(main_release_tmp, main_release_tmp.title.strip())
        # Then we iterate through other versions, mainly to collect new Countries of Release, Catalog #, Year,
        for version_entry in cur_rel.versions:
            update_from_other_release_version(version_entry, track_names, should_update_all, main_release_tmp.title.strip())
    else:
        track_names = get_info_from_release(cur_rel, cur_rel.title.strip())
    print(track_names)


['Funk It (Funkadelala)', 'The Real Thing', 'Stomp!']
["I'm So Excited", 'See How The Love Goes', 'All Of You', 'Heartbeat', 'If You Wanna Get Back Your Lady', 'I Feel For You', 'Heart To Heart', 'American Music', 'Heart Beat', "I'm So Excited = Estoy Tan Exitada", 'See How The Love Goes = Mira Como Va El Amor', 'All Of You = Todo Tú', 'Heartbeat = Latidos', 'If You Wanna Get Back Your Lady = Si Quieres Recuperar Tu Dama', 'I Feel For You = Siento Para Ti', 'Heart To Heart = Corazón A Corazón ', 'American Music = Música Americana', "Estoy Tan Entusiasmada = I'm So Excited", 'Mira Como Va El Amor = See How The Love Goes', 'Todo de Vos = All Of You', 'Latido Del Corazon = Heartbeat', 'Si Quiere Volver A Tener Tu Dama = If You Wanna Get Back Your Lady', 'Yo Siento Por Vos = I Feel For You', 'Corazon A Corazon = Heart To Heart', 'Musica Americana = American Music', 'Estoy Tan Emocionada', 'Mira Como Va El Amor', 'Todo De Ti', 'Latido Del Corazon', 'Si Quieres Regresar A Tu Dama', 'Siento P

KeyboardInterrupt: 

In [6]:
tmp_release = greg_rel_page5[0].main_release

In [ ]:
for tmp_rel in greg_rel:
    if tmp_rel.title == "So Excited!":
        rel_life = tmp_rel
        print(tmp_rel.data["type"])
        break

In [ ]:
master_count = 0
release_count = 0
release_has_master_count = 0
for cur_rel in greg_rel:

    if cur_rel.data["type"] == "master":
        master_count += 1
    else:
        release_count += 1
        if cur_rel.master is not None:
            release_has_master_count += 1

print(master_count, release_count, release_has_master_count)

In [ ]:
import copy
tmp_dic = copy.deepcopy(track_dic)

In [ ]:
for x in track_dic:
    if track_dic[x].requires_manual_check:
        print (track_dic[x].title, track_dic[x].album)

In [ ]:
release8209382 = d.release(8209382)
master8209382 = release8209382.master
artists820 = [artist.name for artist in release8209382.artists]
print(release8209382.credits[6] == greg)

In [ ]:
master_born = d.master(82464)
rel_born = master_born.main_release

In [ ]:
rel1 = release8209382
rel2 = release12059729
master2 = rel2.master
rel3 = d.release(476301)
master3 = rel3
master4 = d.master(261710)
rel4 = master4.main_release

In [ ]:
del track_dic